# Estimación de peso y dimensiones de los envíos de Mercado Libre

Este notebook es una versión mínima y necesaria para poder levantar el dataset [meli_dataset_20190426.csv](https://drive.google.com/open?id=1tNUKD1lf1z8C7LPpCruiDl6SKBAHb79v) y comprender los features que éste contiene.

# Sobre el dataset
El dataset corresponde a un muestreo de envíos durante tres meses de 2018 en MLB (Mercado Libre Brasil), filtrados según métodos de envíos y por carriers (empresas de correos a cargo del envío propiamente dicho). A su vez, fue muestreado de forma no uniforme para romper de forma manual la distribución de los datos.

In [2]:
import pandas as pd
import random

In [3]:
random.seed(0)

In [4]:
DATASET = 'meli_dataset_20190426.csv'

In [5]:
df = pd.read_csv(DATASET, low_memory=False)

In [34]:
df.columns

Index(['ITEM_ID', 'SHP_WEIGHT', 'SHP_LENGTH', 'SHP_WIDTH', 'SHP_HEIGHT',
       'ATTRIBUTES', 'CATALOG_PRODUCT_ID', 'CONDITION', 'DOMAIN_ID', 'PRICE',
       'SELLER_ID', 'STATUS', 'TITLE'],
      dtype='object')

#### ¿Qué significa cada columna?

* ITEM_ID: id unívoco de cada item publicado. (Ofuscado)
* SHP_WEIGHT: peso del paquete informado por el correo.
* SHP_LENGTH: largo del paquete informado por el correo.
* SHP_WIDTH: ancho del paquete informado por el correo.
* SHP_HEIGHT: altura del paquete informado por el correo.
* ATTRIBUTES: atributos como marca y modelo, entre otros, en formato json-lines
* CATALOG_PRODUCT_ID: id del catálogo (ofuscado).
* CONDITION: condición de venta (nuevo o usado).
* DOMAIN_ID: id de la categoría a la que pertenece la publicación.
* PRICE: precio en reales.
* SELLER_ID: id del vendedor (ofuscado).
* STATUS: estado de la publicación (activa, cerrada, pausada, etc.)
* TITLE: título de la publicación.

In [8]:
df.sample(5)

,ITEM_ID,SHP_WEIGHT,SHP_LENGTH,SHP_WIDTH,SHP_HEIGHT,ATTRIBUTES,CATALOG_PRODUCT_ID,CONDITION,DOMAIN_ID,PRICE,SELLER_ID,STATUS,TITLE
181414,Z6TSZIQIEI,3520.0,52.0,47.0,33.0,"[{'id': 'BRAND', 'name': 'Marca', 'value_id': ...",H53U1H7Q5G,new,MLB-VEHICLE_PARTS,119.0,KL8NF8VF5Y,paused,Par Lente Farol Palio Zoião 2004 2005 2006 200...
90027,VGO4R7I9SI,10500.0,79.0,69.0,11.0,"[{'id': 'LINE', 'name': 'Linha', 'value_id': N...",H53U1H7Q5G,new,MLB-CAR_INTERCOOLERS,1090.8,LVY4TGGARR,active,Kit Intercooler Mbb 1113-1313-1513-2013-2213
426038,ESQWLFUBLI,NaN,NaN,NaN,NaN,NaN,H53U1H7Q5G,NaN,NaN,NaN,LSW0UDCLGF,under_review,Pulseira Milanese Apple Watch Loop Magnética -...
138780,ZWZ91HBVZA,NaN,NaN,NaN,NaN,NaN,H53U1H7Q5G,NaN,NaN,NaN,I0CYMA5J3D,under_review,Kit Festa Iluminação 4x1 Balada Bola Strobo La...
400982,SYIK6JKNYQ,484.0,25.0,11.0,8.0,"[{'id': 'BATTERY_CAPACITY', 'name': 'Capacidad...",GP3HBDG9M3,new,MLB-CELLPHONES,1019.0,UTTFJKPUC7,closed,Xiaomi Mi A1 4gb/64gb Lte 5.5 Cam12/12mp Capa ...


In [39]:
# Cosntrucción de bases sin faltantes
#df_wei = df.dropna(subset=['SHP_WEIGHT'])
#df_len = df.dropna(subset=['SHP_LENGTH'])
#df_whi = df.dropna(subset=['SHP_WIDTH'])
#df_hei = df.dropna(subset=['SHP_HEIGHT'])

#df_full = df.dropna(subset=['SHP_WEIGHT'])
#df_full = df_full.dropna(subset=['SHP_LENGTH'])
#df_full = df_full.dropna(subset=['SHP_WIDTH'])
#df_full = df_full.dropna(subset=['SHP_HEIGHT'])
#df_full.sample(3)

df = df.dropna(subset=['SHP_WEIGHT'])
df = df.dropna(subset=['SHP_LENGTH'])
df = df.dropna(subset=['SHP_WIDTH'])
df = df.dropna(subset=['SHP_HEIGHT'])
df.sample(3)


,ITEM_ID,SHP_WEIGHT,SHP_LENGTH,SHP_WIDTH,SHP_HEIGHT,ATTRIBUTES,CATALOG_PRODUCT_ID,CONDITION,DOMAIN_ID,PRICE,SELLER_ID,STATUS,TITLE
216098,M3XXKDKYZ5,1100.0,16.0,11.0,4.0,"[{'id': 'BACKPACK_CAPACITY', 'name': 'Capacida...",H53U1H7Q5G,new,MLB-BACKPACKS,64.9,W8GPQUJYDQ,active,Mochila Cabo Aço Resistente Universitária Esco...
242193,FCU15V6NQG,320.0,20.0,13.0,13.0,"[{'id': 'ITEM_CONDITION', 'name': 'Condição do...",H53U1H7Q5G,new,MLB-FOOTBALL_EQUIPMENT_AND_ACCESSORIES,37.9,Y9P2DX83LN,active,Copo Termico Flamengo Produto Oficial 600 Ml
346863,O46WG19H5W,425.0,25.0,20.0,10.0,NaN,H53U1H7Q5G,NaN,NaN,NaN,BSP892M4YU,under_review,Oculos Realidade Virtual Cardboard 3d Rift Vr ...


El objetivo del análisis es predecir el peso y tamaño de los artículos. A su vez el tamaño se puede descomponer en largo, altura y ancho.  Las cuatro variables a predecir son numéricas continuas .
Probablemente la variable explicativa más importante sea CATALOG_PRODUCT_ID, dado que si se conoce el peso y tamaño de un artículo, los próximos envíos deberían tener las mismas características. 

En el caso de envíos de productos no catalogados previamente, es de esperar que tomen relevancia las variables ATRIBUTES, TITTLE, SELLER_ID, DOMANIN_ID. En el caso de atributes, hay que preprocesar los datoa pra poder extraer algo de información.

Luego se dispone de otras variables complementarias, cuya incorporación podría incrementar el poder predictivo del modelo: CONDITION, STATUS, PRICE.

En primer lugar es conveniente relizar un análisis descriptivo, principalmente de las variables que se desea predecir.

## Analisis estadístico descriptivo de las variables peso, largo, altura y ancho

In [40]:
#df_full.describe()
#df_full[['SHP_WEIGHT', 'SHP_LENGTH', 'SHP_WIDTH', "SHP_HEIGHT"]].describe()
#print(df[['SHP_WEIGHT']].describe() ,
#df[['SHP_LENGTH']].describe(), 
#df[['SHP_WIDTH']].describe() ,
#df[['SHP_HEIGHT']].describe())

df[['SHP_WEIGHT', 'SHP_LENGTH', 'SHP_WIDTH', "SHP_HEIGHT"]].describe()


,SHP_WEIGHT,SHP_LENGTH,SHP_WIDTH,SHP_HEIGHT
count,347751.000000,347751.000000,347751.000000,347751.000000
mean,1854.789579,31.451148,21.290276,11.578580
std,3257.884009,18.339895,11.217308,8.310821
min,1.000000,0.000000,0.000000,0.000000
25%,260.000000,20.000000,13.000000,5.000000
50%,675.000000,25.000000,20.000000,10.000000
75%,1900.000000,36.000000,25.000000,16.000000
max,50000.000000,288.200000,165.000000,105.000000
